In [274]:
%cd /home/stefano/dev/active/spreads-arb
%load_ext line_profiler

import itertools
import logging
import os
import pickle
from datetime import date, datetime, timedelta

import cryptomart as cm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pyutil
import requests
import vectorbt as vbt

import argparse
import os
import glob
import pickle

import cryptomart as cm
import pandas as pd
from app.data_prep.dummy import dummy_bid_ask_spreads
from app.data_prep.real import all_bid_ask_spreads, all_funding_rates, all_ohlcv, get_fee_info
from app.feeds import Spread
import app
from IPython.display import display, HTML
from functools import cached_property
from app.data_prep.tardis_download import TardisData
from app.analysis_tools import plotting

identifier = "spread-arb-v2"
# app.data_prep.real.aggregate_tardis_quotes()


/home/stefano/dev/active/spreads-arb
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Tick by tick quote data from April 10 to May 10 exists in /home/stefano/dev/active/spreads-arb/data/tick_quotes

Aggregate quote data into hourly bars. For each hour take the first available bid/ask price and take the maximum bid/ask size within the first 8 seconds

In [214]:
in_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes"
out_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg"


def aggregate_quotes(df):
    if df.empty:
        return None

    start = df.timestamp.iloc[0].replace(second=0, microsecond=0)

    threshold = df[df.timestamp <= start + pd.Timedelta(seconds=8)]

    if threshold.empty:
        return None

    return pd.Series(
        {
            "bid_price": threshold.bid_price.iloc[0],
            "bid_amount": threshold.bid_amount.max(),
            "ask_price": threshold.ask_price.iloc[0],
            "ask_amount": threshold.ask_amount.max(),
        }
    )


for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=True):
    df = pd.read_parquet(filepath)
    df = df.groupby(pd.Grouper(key="timestamp", freq="1h")).apply(aggregate_quotes).reset_index()
    os.makedirs(os.path.join(out_path, exchange), exist_ok=True)
    df.to_parquet(os.path.join(out_path, exchange, symbol + ".parquet"))

Progress: 0.00%
Progress: 10.00%
Progress: 20.00%
Progress: 40.00%
Progress: 50.00%
Progress: 80.00%
Progress: 90.00%
Progress: 100.00%


In [402]:
x = pd.read_parquet("/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg/okex/GFT.parquet")

In [403]:
x

,timestamp,bid_price,bid_amount,ask_price,ask_amount,filled
0,2023-04-10 00:00:00,0.0429,1445.0,0.0430,170.0,0
1,2023-04-10 01:00:00,0.0428,572.0,0.0429,881.0,0
2,2023-04-10 02:00:00,0.0425,383.0,0.0426,1417.0,0
3,2023-04-10 03:00:00,0.0427,827.0,0.0428,788.0,0
4,2023-04-10 04:00:00,0.0429,1816.0,0.0430,1583.0,0
...,...,...,...,...,...,...
715,2023-05-09 19:00:00,0.0254,3549.0,0.0255,496.0,0
716,2023-05-09 20:00:00,0.0256,2693.0,0.0257,2809.0,0
717,2023-05-09 21:00:00,0.0252,7690.0,0.0253,1901.0,0
718,2023-05-09 22:00:00,0.0255,1520.0,0.0256,1057.0,0


Apply forward filling

In [218]:
in_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg"

quotes_stats = app.data_prep.tardis_download.create_exchange_dataframe()
for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=False):
    df = pd.read_parquet(filepath)
    if "filled" in df.columns:
        continue
    df["filled"] = df.bid_amount.isna().astype(int)
    df = df.ffill().bfill()
    df.to_parquet(filepath)

Get stats for 1h quotes

In [238]:
in_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg"

quotes_stats = app.data_prep.tardis_download.create_exchange_dataframe()
for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=False):
    df = pd.read_parquet(filepath)
    quotes_stats.at[(exchange, symbol), "earliest_time"] = df.timestamp.min()
    quotes_stats.at[(exchange, symbol), "latest_time"] = df.timestamp.max()
    quotes_stats.at[(exchange, symbol), "total_rows"] = df.shape[0]
    quotes_stats.at[(exchange, symbol), "invalid_rows"] = df.filled.sum()
    quotes_stats.at[(exchange, symbol), "valid_rows"] = (1 - df.filled).sum()
    
quotes_stats.sort_values("invalid_rows")


earliest_time         latest_time  total_rows  invalid_rows  \
exchange symbol                                                               
okex     OMG       2023-04-10 2023-05-09 23:00:00       720.0           0.0   
gateio   LTC       2023-04-10 2023-05-09 23:00:00       720.0           0.0   
okex     MANA      2023-04-10 2023-05-09 23:00:00       720.0           0.0   
         EOS       2023-04-10 2023-05-09 23:00:00       720.0           0.0   
         TRX       2023-04-10 2023-05-09 23:00:00       720.0           0.0   
...                       ...                 ...         ...           ...   
gateio   NFT       2023-04-10 2023-05-09 23:00:00       720.0         504.0   
bitmex   ARB       2023-04-10 2023-05-09 23:00:00       720.0         615.0   
         GMX       2023-04-10 2023-05-09 23:00:00       720.0         635.0   
gateio   BUSD      2023-04-10 2023-05-09 21:00:00       718.0         651.0   
bitmex   BLUR      2023-04-10 2023-05-09 20:00:00       717.0         655.0   

                 valid_rows  
exchange symbol              
okex     OMG          720.0  
gateio   LTC          720.0  
okex     MANA         720.0  
         EOS          720.0  
         TRX          720.0  
...                     ...  
gateio   NFT          216.0  
bitmex   ARB          105.0  
         GMX           85.0  
gateio   BUSD          67.0  
bitmex   BLUR          62.0  

[703 rows x 5 columns]

Get daily close prices for the first 30 days of each symbol

In [296]:
in_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg"
out_path = "/home/stefano/dev/active/spreads-arb/data/daily_close"

for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=False):
    outfile = os.path.join(out_path, exchange, symbol + ".parquet")
    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    ohlcv = app.data_prep.real.cm_client.ohlcv(exchange, symbol, "perpetual", "2023-03-11", "2023-04-10", cache_kwargs={"log_level": "DEBUG"})
    daily_close = ohlcv.set_index("open_time")[["close"]]
    daily_close["filled"] = daily_close.close.isna().astype(int)
    daily_close = daily_close.ffill().bfill()
    daily_close.to_parquet(outfile)


Get stats for daily quotes

In [299]:
in_path = "/home/stefano/dev/active/spreads-arb/data/daily_close"

daily_close_stats = app.data_prep.tardis_download.create_exchange_dataframe()
for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=False):
    df = pd.read_parquet(filepath)
    daily_close_stats.at[(exchange, symbol), "earliest_time"] = df.index.min()
    daily_close_stats.at[(exchange, symbol), "latest_time"] = df.index.max()
    daily_close_stats.at[(exchange, symbol), "total_rows"] = df.shape[0]
    daily_close_stats.at[(exchange, symbol), "invalid_rows"] = df.filled.sum()
    daily_close_stats.at[(exchange, symbol), "valid_rows"] = (1 - df.filled).sum()
    
daily_close_stats.sort_values("invalid_rows")

earliest_time latest_time  total_rows  invalid_rows  \
exchange symbol                                                       
okex     GFT       2023-03-11  2023-04-09        30.0           0.0   
bybit    DODO      2023-03-11  2023-04-09        30.0           0.0   
         COMP      2023-03-11  2023-04-09        30.0           0.0   
         CKB       2023-03-11  2023-04-09        30.0           0.0   
         TRU       2023-03-11  2023-04-09        30.0           0.0   
...                       ...         ...         ...           ...   
         JOE       2023-03-11  2023-04-09        30.0          18.0   
gateio   RDNT      2023-03-11  2023-04-09        30.0          19.0   
binance  TLM       2023-03-11  2023-04-09        30.0          19.0   
         RDNT      2023-03-11  2023-04-09        30.0          25.0   
         HFT       2023-03-11  2023-04-09        30.0          26.0   

                 valid_rows  
exchange symbol              
okex     GFT           30.0  
bybit    DODO          30.0  
         COMP          30.0  
         CKB           30.0  
         TRU           30.0  
...                     ...  
         JOE           12.0  
gateio   RDNT          11.0  
binance  TLM           11.0  
         RDNT           5.0  
         HFT            4.0  

[703 rows x 5 columns]

Keep all symbols that have at least 30 days of daily data and less than 30 hours missing of hourly data

In [300]:
keep_symbols = daily_close_stats[daily_close_stats.invalid_rows == 0].join(quotes_stats[quotes_stats.invalid_rows < 30], lsuffix="_x", rsuffix="_y", how="inner").index
len(keep_symbols)

519

Append daily close data to hourly quotes and create price_signal as bid/ask midpoint

In [301]:
quotes_path = "/home/stefano/dev/active/spreads-arb/data/tick_quotes_1h_agg"
daily_close_path = "/home/stefano/dev/active/spreads-arb/data/daily_close"
out_path = "/home/stefano/dev/active/spreads-arb/data/price_signal_1h"

for (exchange, symbol, quotes_fn), (_, __, daily_close_fn) in zip(
    app.data_prep.get_cryptomart_data_iterator(quotes_path, keep_symbols),
    app.data_prep.get_cryptomart_data_iterator(daily_close_path, keep_symbols, show_progress=False),
):
    quotes = pd.read_parquet(quotes_fn)
    daily_close = pd.read_parquet(daily_close_fn)
    quotes["price_signal"] = quotes.eval("(ask_price + bid_price) / 2")
    daily_close = (
        daily_close.reindex(pd.date_range(daily_close.index.min(), quotes.timestamp.min(), freq="1h")[:-1])
        .ffill()
        .bfill()
        .rename(columns={"close": "price_signal"})
        .astype({"filled": int})
        .rename_axis("timestamp")
        .reset_index()
    )
    price_signal = pd.concat([daily_close, quotes])
    outfile = os.path.join(out_path, exchange, symbol + ".parquet")
    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    price_signal.to_parquet(outfile)

Load price signal data into memory

In [308]:
in_path = "/home/stefano/dev/active/spreads-arb/data/price_signal_1h"

class DataFrameHolder:
    def __init__(self, df, name=""):
        self.df = df
        self.name = name
        
    def __repr__(self):
        return f"DFH({self.df.shape}, {self.df.columns.tolist()})"
        
    def get(self):
        return self.df

price_signal_dfs = app.data_prep.tardis_download.create_exchange_dataframe()
for exchange, symbol, filepath in app.data_prep.get_cryptomart_data_iterator(in_path, show_progress=False):
    price_signal_dfs.at[(exchange, symbol), "df"] = DataFrameHolder(pd.read_parquet(filepath), name=f"{exchange}.{symbol}")
    
len(price_signal_dfs)


519

Create Spreads

In [551]:

try:
    profile
except NameError:
    profile = lambda x: x

@profile
def compute_signals(
    spread,
    price_column="price_signal",
    z_score_period=pd.Timedelta(days=30),
    z_score_thresh=1.0,
    remove_intermediate_cols=True,
):
    data_frequency = spread.timestamp.diff().mean()
    row_period = z_score_period / data_frequency
    assert row_period == int(row_period), f"z_score_period must be a multiple of data_frequency ({data_frequency})"

    rolling_window = spread[price_column].rolling(int(row_period))
    spread["zscore"] = (spread[price_column] - rolling_window.mean()) / rolling_window.std()
    crossed = ((spread["zscore"] * spread["zscore"].shift(1)) <= 0).astype(int)
    long_signal = (spread["zscore"] < -z_score_thresh).astype(int)
    short_signal = (spread["zscore"] > z_score_thresh).astype(int)
    long_hold = long_signal.groupby(crossed.cumsum()).cummax()
    short_hold = short_signal.groupby(crossed.cumsum()).cummax()
    spread["long_entry"] = (long_hold.diff() == 1).astype(int)
    spread["long_exit"] = (long_hold.diff() == -1).astype(int)
    spread["short_entry"] = (short_hold.diff() == 1).astype(int)
    spread["short_exit"] = (short_hold.diff() == -1).astype(int)

    long_spread_trade = (spread.long_entry | spread.short_exit).astype(bool)
    short_spread_trade = (spread.short_entry | spread.long_exit).astype(bool)
   
    # Long spread means leg_x is sold (bid) and leg_y is bought (ask)
    spread.loc[long_spread_trade, "real_price"] = spread.loc[long_spread_trade].eval("ask_price_y - bid_price_x")
    # short spread means leg_x is bought (ask) and leg_y is sold (bought)
    spread.loc[short_spread_trade, "real_price"] = spread.loc[short_spread_trade].eval("bid_price_y - ask_price_x")
   
    # apply padding to the amount columns
    long_spread_trade = (long_spread_trade | long_spread_trade.shift(1) | long_spread_trade.shift(-1))
    short_spread_trade = (short_spread_trade | short_spread_trade.shift(1) | short_spread_trade.shift(-1))
    spread.loc[long_spread_trade, "real_amount"] = spread.loc[long_spread_trade, ["ask_amount_y", "bid_amount_x"]].min(
        axis=1
    )
    spread.loc[short_spread_trade, "real_amount"] = spread.loc[
        short_spread_trade, ["ask_amount_x", "bid_amount_y"]
    ].min(axis=1)

    if remove_intermediate_cols:
        spread.drop(
            columns=[
                x for x in spread.columns if (x.endswith("_x") or x.endswith("_y")) and not x.startswith("filled")
            ],
            inplace=True,
        )

def get_trades(spread):
    def group_trade(g, side="long"):
        if len(g) < 2:
            return None
        
        side_multiplier = 1 if side == "long" else -1
        
        entry_price = g.iloc[0].real_price
        exit_price = g.iloc[1].real_price
        amount = g.real_amount.min()
        entry_time = g.iloc[0].timestamp
        exit_time = g.iloc[1].timestamp
        duration = exit_time - entry_time
        entry_zscore = g.iloc[0].zscore
        exit_zscore = g.iloc[1].zscore
        z_score_movement = abs(exit_zscore - entry_zscore)
        raw_profit = side_multiplier * (exit_price - entry_price) * amount
        transacted_value = (abs(entry_price) + abs(exit_price)) * amount
        fees_paid = transacted_value * 0.00075
        profit = raw_profit - fees_paid
        
        return pd.Series({
            "side": side,
            "entry_time": entry_time,
            "exit_time": exit_time,
            "duration": duration,
            "entry_price": entry_price,
            "exit_price": exit_price,
            "amount": amount,
            "entry_zscore": entry_zscore,
            "exit_zscore": exit_zscore,
            "z_score_movement": z_score_movement,
            "raw_profit": raw_profit,
            "transacted_value": transacted_value,
            "fees_paid": fees_paid,
            "profit": profit,
        })

    long_filter = spread[(spread.long_entry | spread.long_exit).astype(bool)]
    short_filter = spread[(spread.short_entry | spread.short_exit).astype(bool)]
    long_trades = long_filter.groupby(long_filter.reset_index(drop=True).index // 2).apply(group_trade, side="long")
    short_trades = short_filter.groupby(short_filter.reset_index(drop=True).index // 2).apply(group_trade, side="short")
    trades = pd.concat([long_trades, short_trades]).sort_values("entry_time").reset_index(drop=True)
    return trades
    

def trades_summary(spread):
    trades = get_trades(spread)
    return pd.Series({
        "avg_profit": trades.profit.mean(),
        "best_profit": trades.profit.max(),
        "worst_profit": trades.profit.min(),
        "avg_duration": trades.duration.mean(),
        "avg_transacted_value": trades.transacted_value.mean(),
        "avg_fees_paid": trades.fees_paid.mean(),
        "highest_z_score_movement": trades.z_score_movement.max(),
        "lowest_z_score_movement": trades.z_score_movement.min(),
        "avg_z_score_movement": trades.z_score_movement.mean(),
    })

class SpreadHolder(DataFrameHolder):
    def __init__(self, leg_x, leg_y, **kwargs):
        spread = leg_x.merge(leg_y, on="timestamp")
        spread["bid_price"] = spread.eval("bid_price_y - bid_price_x")
        spread["ask_price"] = spread.eval("ask_price_y - ask_price_x")
        spread["price_signal"] = spread.eval("price_signal_y - price_signal_x")
        spread["bid_amount"] = spread.eval("(bid_amount_y + bid_amount_x) / 2")
        spread["ask_amount"] = spread.eval("(ask_amount_y + ask_amount_x) / 2")

        compute_signals(spread)

        super().__init__(spread, **kwargs)

        self.leg_x = leg_x
        self.leg_y = leg_y

    def get_x(self):
        return self.leg_x

    def get_y(self):
        return self.leg_y

    def plot(self):
        plotting.plot_df_subplots(
            df=self.df,
            x="timestamp",
            y=[
                ["price_signal", "zscore"],
                ["real_amount"],
                ["real_price", "long_entry", "long_exit", "short_entry", "short_exit"],
                ["filled_x", "filled_y"],
            ],
            row_heights=[0.5, 0.2, 0.2, 0.2],
            multi_y=[["zscore"], [], ["real_price"], []],
            scatter_kwargs=[
                [{}, {"opacity": 0.4}],
            ],
            update_layout_kwargs={"title": self.name},
        )

In [502]:
ps_cross_product = (
    price_signal_dfs.reset_index()
    .merge(price_signal_dfs.reset_index(), how="cross", suffixes=("_x", "_y"))
    .pipe(lambda df: df[(df.exchange_x < df.exchange_y) & (df.symbol_x == df.symbol_y)])
    .drop(columns="symbol_y")
    .rename(columns={"symbol_x": "symbol"})
    .reset_index(drop=True)
)[["symbol", "exchange_x", "exchange_y", "df_x", "df_y"]].set_index(["symbol", "exchange_x", "exchange_y"])

spreads = ps_cross_product.apply(
    lambda row: SpreadHolder(row.df_x.get(), row.df_y.get(), name=f"{row.df_y.name} - {row.df_x.name}"), axis=1
)
spreads

symbol  exchange_x  exchange_y
FIL     gateio      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
BLUR    gateio      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
LUNC    gateio      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
LINK    gateio      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
DOGE    gateio      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
                                                        ...                        
LTC     bitmex      gateio        DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
                    bybit         DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
BTC     bitmex      okex          DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
                    gateio        DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
                    bybit         DFH((1440, 15), ['timestamp', 'filled_x', 'fil...
Length: 512, dtype: object

In [553]:
spreads_trades_summary = spreads.apply(lambda x: trades_summary(x.get()))

In [563]:
spreads_trades_summary.round(2).sort_values("avg_profit", ascending=False).head(20)

avg_profit  best_profit  worst_profit  \
symbol exchange_x exchange_y                                          
BTC    bitmex     gateio      2309936.03  10220112.96     211437.30   
                  okex          33007.76    229457.58       1226.71   
       gateio     okex          11970.56     61961.91       1543.19   
       bitmex     bybit          2117.85     21951.46         16.03   
ETH    bybit      okex           1517.80     16840.34         32.63   
       gateio     okex           1373.58      5233.92        127.49   
       bybit      gateio         1055.38      8600.16         22.51   
BTC    binance    bitmex          401.80      1494.12         34.68   
BNB    gateio     okex            386.51      1828.67          1.36   
LTC    bitmex     gateio          248.86      1180.44       -189.68   
BTC    binance    gateio          171.98      1884.32         28.92   
       bybit      okex            171.44       501.51         26.94   
       binance    bybit           167.52      1915.02         22.66   
                  okex            162.26      1064.26         20.20   
       bybit      gateio          161.62       429.77         21.39   
LTC    bitmex     bybit           161.29      1057.01        -70.21   
XRP    binance    bybit           145.84      4912.86          0.00   
MASK   binance    bybit           139.89      1394.18          0.69   
ETH    binance    okex            120.96       281.04         35.35   
MATIC  bitmex     bybit           106.25       564.53          9.18   

                                          avg_duration  avg_transacted_value  \
symbol exchange_x exchange_y                                                   
BTC    bitmex     gateio     0 days 06:25:23.076923076            2552345.34   
                  okex       0 days 05:10:23.076923076              46179.13   
       gateio     okex       0 days 04:03:33.861386138              38495.69   
       bitmex     bybit      0 days 03:19:24.705882352               2148.46   
ETH    bybit      okex                 0 days 15:30:00               1605.56   
       gateio     okex                 0 days 04:43:20               4238.23   
       bybit      gateio               0 days 09:23:20               1076.65   
BTC    binance    bitmex     0 days 05:27:55.862068965                450.61   
BNB    gateio     okex       0 days 06:58:53.333333333                541.54   
LTC    bitmex     gateio     0 days 03:45:52.941176470                318.10   
BTC    binance    gateio               0 days 03:57:30                176.85   
       bybit      okex       0 days 06:02:43.636363636                398.22   
       binance    bybit      0 days 05:44:34.285714285                169.96   
                  okex                 0 days 03:25:00                465.45   
       bybit      gateio               0 days 05:11:15                161.74   
LTC    bitmex     bybit      0 days 04:21:49.090909090                185.22   
XRP    binance    bybit                0 days 02:52:30                152.63   
MASK   binance    bybit                0 days 03:00:00                141.14   
ETH    binance    okex                 0 days 07:45:00                268.92   
MATIC  bitmex     bybit      0 days 07:44:12.631578947                116.27   

                              avg_fees_paid  highest_z_score_movement  \
symbol exchange_x exchange_y                                            
BTC    bitmex     gateio            1914.26                      7.61   
                  okex                34.63                      7.07   
       gateio     okex                28.87                      6.55   
       bitmex     bybit                1.61                     26.51   
ETH    bybit      okex                 1.20                     26.50   
       gateio     okex                 3.18                      5.81   
       bybit      gateio               0.81                     27.61   
BTC    binance    bitmex               0.34      

In [ ]:
spreads.iloc[29].plot()

In [542]:
get_trades(spreads.iloc[29].get()).sort_values("profit", ascending=False).head(20)

,side,entry_time,exit_time,duration,entry_price,exit_price,amount,entry_zscore,exit_zscore,z_score_movement,raw_profit,transacted_value,fees_paid,profit
9,short,2023-04-13 09:00:00,2023-04-13 14:00:00,0 days 05:00:00,24.1,11.1,4776.0,2.194798,-0.807123,-3.001922,62088.0,168115.2,126.086400,61961.913600
22,long,2023-04-16 18:00:00,2023-04-16 19:00:00,0 days 01:00:00,9.5,29.4,2189.0,-1.145247,3.220279,4.365526,43561.1,85152.1,63.864075,43497.235925
28,short,2023-04-18 12:00:00,2023-04-18 16:00:00,0 days 04:00:00,44.9,14.4,1315.0,6.418689,-0.132284,-6.550974,40107.5,77979.5,58.484625,40049.015375
12,long,2023-04-13 21:00:00,2023-04-13 23:00:00,0 days 02:00:00,7.0,18.5,3050.0,-1.785160,0.926463,2.711623,35075.0,77775.0,58.331250,35016.668750
58,short,2023-04-28 20:00:00,2023-04-28 21:00:00,0 days 01:00:00,23.3,14.1,3504.0,1.922446,-0.122967,-2.045413,32236.8,131049.6,98.287200,32138.512800
21,short,2023-04-16 11:00:00,2023-04-16 18:00:00,0 days 07:00:00,22.6,9.5,2220.0,1.766705,-1.145247,-2.911952,29082.0,71262.0,53.446500,29028.553500
27,long,2023-04-18 09:00:00,2023-04-18 10:00:00,0 days 01:00:00,6.2,14.8,3268.0,-1.889955,0.026768,1.916723,28104.8,68628.0,51.471000,28053.329000
2,long,2023-04-11 04:00:00,2023-04-11 09:00:00,0 days 05:00:00,8.5,17.0,2656.0,-1.185181,0.612456,1.797637,22576.0,67728.0,50.796000,22525.204000
23,short,2023-04-16 19:00:00,2023-04-16 20:00:00,0 days 01:00:00,29.4,12.0,1231.0,3.220279,-0.597117,-3.817396,21419.4,50963.4,38.222550,21381.177450
59,long,2023-04-28 22:00:00,2023-04-28 23:00:00,0 days 01:00:00,6.0,16.5,2018.0,-1.878695,0.445094,2.323789,21189.0,45405.0,34.053750,21154.946250


In [ ]:
spreads.iloc[0].plot()

In [ ]:
spreads.iloc[1].plot()

In [ ]:
plotting.plot_df(x, "timestamp", ["price_signal", "zscore"], multi_y=["zscore"])

In [ ]:
plotting.plot_df_subplots(
    x,
    "timestamp",
    [
        ["price_signal", "zscore", "real_price"],
        ["long_signal", "short_signal"],
        ["long_hold", "short_hold"],
        ["long_entry", "long_exit", "short_entry", "short_exit"],
    ],
    row_heights=[0.6, 0.2, 0.2, 0.2],
    multi_y=[
        ["zscore"], [], [], [],
    ]
)

# plotting.plot_df(x, "timestamp", ["price_signal", "zscore", "crossed"], multi_y=["zscore", "crossed"])

In [364]:
x.diff()

,timestamp,bid_price,bid_amount,ask_price,ask_amount,price_signal,zscore,real_price,real_amount,filled_x,filled_y,crossed,long_signal,short_signal,test,test_1,long_hold,short_hold
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0 days 01:00:00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0 days 01:00:00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0 days 01:00:00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0 days 01:00:00,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0 days 01:00:00,0.002,18361.0,2.000000e-03,-60091.0,2.000000e-03,1.036979,NaN,NaN,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
1436,0 days 01:00:00,-0.001,12888.0,-1.000000e-03,90601.5,-1.000000e-03,-0.517527,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1437,0 days 01:00:00,0.000,-3771.5,8.881784e-16,-105906.0,0.000000e+00,0.000961,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438,0 days 01:00:00,0.000,-45945.5,0.000000e+00,-10872.5,-8.881784e-16,0.000960,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [371]:
pd.Timedelta(days=30) / x.timestamp.diff().mean()

720.0

In [363]:
x.shift(1)

,timestamp,bid_price,bid_amount,ask_price,ask_amount,price_signal,zscore,real_price,real_amount,filled_x,filled_y,crossed,long_signal,short_signal,test,test_1,long_hold,short_hold
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-11 00:00:00,NaN,NaN,NaN,NaN,0.011,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-03-11 01:00:00,NaN,NaN,NaN,NaN,0.011,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-03-11 02:00:00,NaN,NaN,NaN,NaN,0.011,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-03-11 03:00:00,NaN,NaN,NaN,NaN,0.011,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2023-05-09 18:00:00,0.001,42950.5,0.001,110699.0,0.001,-1.242574,0.0,44440.0,0.0,0.0,0.0,1.0,0.0,1.0,280.0,1.0,0.0
1436,2023-05-09 19:00:00,0.003,61311.5,0.003,50608.0,0.003,-0.205595,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,280.0,1.0,0.0
1437,2023-05-09 20:00:00,0.002,74199.5,0.002,141209.5,0.002,-0.723122,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,280.0,1.0,0.0
1438,2023-05-09 21:00:00,0.002,70428.0,0.002,35303.5,0.002,-0.722161,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,280.0,1.0,0.0


In [357]:
x.groupby(x.crossed.cumsum()).apply(lambda g: print(g))

              timestamp  bid_price  bid_amount  ask_price  ask_amount  \
0   2023-03-11 00:00:00        NaN         NaN        NaN         NaN   
1   2023-03-11 01:00:00        NaN         NaN        NaN         NaN   
2   2023-03-11 02:00:00        NaN         NaN        NaN         NaN   
3   2023-03-11 03:00:00        NaN         NaN        NaN         NaN   
4   2023-03-11 04:00:00        NaN         NaN        NaN         NaN   
..                  ...        ...         ...        ...         ...   
718 2023-04-09 22:00:00        NaN         NaN        NaN         NaN   
719 2023-04-09 23:00:00        NaN         NaN        NaN         NaN   
720 2023-04-10 00:00:00      0.003     24660.5      0.003     49401.0   
721 2023-04-10 01:00:00      0.001     50200.0      0.001     43743.5   
722 2023-04-10 02:00:00      0.002     18243.0      0.002     32769.0   

     price_signal    zscore  real_price  real_amount  filled_x  filled_y  \
0           0.011       NaN         NaN        

""


In [339]:
x.price_signal.shift(1)

0         NaN
1       0.011
2       0.011
3       0.011
4       0.011
        ...  
1435    0.001
1436    0.003
1437    0.002
1438    0.002
1439    0.002
Name: price_signal, Length: 1440, dtype: float64

In [ ]:
plotting.plot_df(x, "timestamp", ["price_signal", "zscore", "crossed"], multi_y=["zscore", "crossed"])

In [145]:
x = pd.read_parquet("/home/stefano/dev/active/spreads-arb/data/price_signal_1h/binance/1000LUNC.parquet")

In [146]:
x.filled.sum()

4

In [ ]:
window = x.price_signal.rolling(24*30)
x["zscore"] = (x.price_signal - window.mean()) / window.std()

plotting.plot_df_multi_y(x, "timestamp", "price_signal", "zscore")